In [26]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split

# Download latest version
path = kagglehub.dataset_download("kukuroo3/body-signal-of-smoking")

df = pd.read_csv(path+"/smoking.csv")
df = df.drop(columns = ["oral", "ID"])
df["tartar"] = df["tartar"].map({"Y": 0, "N": 1})
df["gender"] = df["gender"].map({"F": 0, "M": 1})

In [30]:
# split off check set
df_work, df_check = train_test_split(
    df, 
    test_size=5000,
    stratify=df["smoking"],
    random_state=5510
)
df = df_work

In [31]:
def equalise_set(dataframe):

    #equalise smokers per gender
    
    df_f = dataframe[dataframe["gender"] == 0]
    df_m = dataframe[dataframe["gender"] == 1]
    
    
    f_smoke = df_f["smoking"].value_counts().max()
    
    df_f_equal = (
        df_f.groupby("smoking")
            .sample(n=f_smoke, replace=True, random_state=5510)
            .reset_index(drop=True)
    )
    
    
    m_smoke = df_m["smoking"].value_counts().max()
    
    df_m_equal = (
        df_m.groupby("smoking")
            .sample(n=m_smoke, replace=True, random_state=5510)
            .reset_index(drop=True)
    )
    
    dataframe = pd.concat([df_f_equal, df_m_equal], axis=0).reset_index(drop=True)
    
    #equalise gender
    
    gender_count = dataframe["gender"].value_counts().max()
    
    df_gender_equal = (
        df.groupby("gender")
            .sample(n=gender_count, replace=True, random_state=1)
            .reset_index(drop=True)
    )
    
    return dataframe
df = equalise_set(df)
df_check = equalise_set(df_check)

In [32]:
# print to csv
# call this to print a final dataset 

df.to_csv("./dataset/dataset.csv", index=False)

df_check.to_csv("./dataset/check.csv", index=False)

In [35]:
print(df.std())

gender                  0.481250
age                    12.071418
height(cm)              9.194597
weight(kg)             12.820306
waist(cm)               9.274223
eyesight(left)          0.486873
eyesight(right)         0.485964
hearing(left)           0.157902
hearing(right)          0.159564
systolic               13.675989
relaxation              9.679278
fasting blood sugar    20.795591
Cholesterol            36.297940
triglyceride           71.639817
HDL                    14.738963
LDL                    40.926476
hemoglobin              1.564498
Urine protein           0.404882
serum creatinine        0.221524
AST                    19.355460
ALT                    30.947853
Gtp                    50.290539
dental caries           0.409665
tartar                  0.496908
smoking                 0.482070
dtype: float64


In [19]:
print(df.mean())

gender                   0.501858
age                     44.594123
height(cm)             162.954501
weight(kg)              63.865472
waist(cm)               81.026310
eyesight(left)           0.987006
eyesight(right)          0.987169
hearing(left)            1.025726
hearing(right)           1.027316
systolic               120.165588
relaxation              75.324350
fasting blood sugar     99.304774
Cholesterol            197.474583
triglyceride           126.370046
HDL                     58.753546
LDL                    113.970249
hemoglobin              14.384044
Urine protein            1.086650
serum creatinine         0.853925
AST                     25.520421
ALT                     25.510006
Gtp                     38.965422
dental caries            0.217533
tartar                   0.426790
smoking                  0.500000
dtype: float64


In [27]:
#introduce blood pressure difference
df["pulse_pressure"] = df["systolic"] - df["relaxation"]

In [28]:
#introduce MAP (mean arterial pressure)
df["map"] = df["relaxation"] + (df["systolic"] - df["relaxation"]) / 3

In [29]:
#introduce BMI and deurenberg formula
df["BMI"] = df["weight(kg)"] / (df["height(cm)"] * df["height(cm)"] / 10000)

df["Deurenberg"] = (1.2*df["BMI"]) + (0.23 * df["age"]) - (10.8 * df["gender"]) - 5.4

In [62]:
import numpy as np

#triglycerides feature engineering
df["log_triglyceride"] = np.log1p(df["triglyceride"])

df["tri_hdl_ratio"] = df["triglyceride"] / df["HDL"]
df["cho_hdl_ratio"] = df["Cholesterol"] / df["HDL"]

df["ldl_hdl_ratio"] = df["LDL"] / df["HDL"]

In [63]:
# "non-hdl" feature
df["non_HDL"] = df["Cholesterol"] - df["HDL"]